# WeatherPy

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [2]:
# Save config information
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "metric"

# Build query URL
query_url = f"{url}appid={weather_api_key}&units={units}&q="

# List to hold responses (storing name, lat dups for simplicity of moving into dictionary)
city_num = []
city_name = []
lat = []
lng = []
temp = []
humidity = []
clouds = []
wind_speed = []

# Loop through cities list to perform a request for info on each
for city in cities:
    
    # Create query, make request, and store in json
    weather_response = requests.get(query_url + city).json()
    
    # Grab data where available
    try:
        city_num.append(weather_response['id'])
        city_name.append(weather_response['name'])
        lat.append(weather_response['coord']['lat'])
        lng.append(weather_response['coord']['lng'])
        temp.append(weather_response['main']['temp'])
        humidity.append(weather_response['main']['humidity'])
        clouds.append(weather_response['clouds']['all'])
        wind_speed.append(weather_response['wind']['speed'])
    
    # If any fields are blank, do not append to lists
    except:
        pass
    
    # Add interval between queries to stay within API query limits
    time.sleep(1)

    for x in city_num:
        # Print log
        print(f"The weather API responded with: {city_num},{city_name}.")

610

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [ ]:
# Create dictionary to hold response data for conversion into dataframe
weather_dict = {
    "city id": city_num,
    "city": city_name,
    "lat": lat,
    "temp": temp,
    "humidity": humidity,
    "cloudiness": clouds,
    "wind speed": wind_speed    
}

# Create dataframe
cities_df = pd.DataFrame(weather_dict)

# Display dataframe
cities_df.head()

In [ ]:
cities_df.to_csv("cities.csv")

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
# Import cities.csv so API doesn't need to be called again to work with data
final_cities = "cities.csv"

final_cities_df = pd.read_csv(final_cities)
final_cities_df.drop(["Unnamed: 0"], axis=1, inplace=True)

final_cities_df.head()

In [ ]:
# Get the indices of cities that have humidity over 100%.
humidity_error = final_cities_df.loc[final_cities_df["humidity"] > 100]
humidity_error

## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

In [ ]:
plt.scatter(final_cities_df.iloc[:,2],final_cities_df.iloc[:,3])
plt.xlabel("Latitude")
plt.ylabel("Temperature (C)")
plt.title("Latitude vs. Temperature")
plt.show()

# Save as .png
plt.savefig('lat_temp.png')

## Latitude vs. Humidity Plot

In [ ]:
plt.scatter(final_cities_df.iloc[:,2],final_cities_df.iloc[:,4])
plt.xlabel("Latitude")
plt.ylabel("Humidity")
plt.title("Latitude vs. Humidity")
plt.show()

# Save as .png
plt.savefig('lat_hum.png')

## Latitude vs. Cloudiness Plot

In [ ]:
plt.scatter(final_cities_df.iloc[:,2],final_cities_df.iloc[:,5])
plt.xlabel("Latitude")
plt.ylabel("Cloudiness")
plt.title("Latitude vs. Cloudiness")
plt.show()

# Save as .png
plt.savefig('lat_clouds.png')

## Latitude vs. Wind Speed Plot

In [ ]:
plt.scatter(final_cities_df.iloc[:,2],final_cities_df.iloc[:,6])
plt.xlabel("Latitude")
plt.ylabel("Wind Speed")
plt.title("Latitude vs. Wind Speed")
plt.show()

# Save as .png
plt.savefig('lat_wind.png')

## Linear Regression

In [ ]:
# Create northern hemisphere dataframe
northern_hem = final_cities_df[["city id", "city", "lat", "temp", "humidity", "cloudiness",
                                "wind speed"]].loc[(final_cities_df["lat"] >= 0)]

# Create southern hemisphere dataframe
southern_hem = final_cities_df[["city id", "city", "lat", "temp", "humidity", "cloudiness",
                                "wind speed"]].loc[(final_cities_df["lat"] <= 0)]

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
# Create scatter plot
plt.scatter(northern_hem.iloc[:,2],northern_hem.iloc[:,3])
plt.xlabel("Latitude")
plt.ylabel("Temperature (C)")
plt.title("Latitude vs. Temperature, Northern Hemisphere")

# Calculate and apply linear regression
(slope, intercept, rvalue, pvalue, stderr) = linregress(northern_hem.iloc[:,2],northern_hem.iloc[:,3])
regress_values_north_temp = northern_hem.iloc[:,2] * slope + intercept
line_eq_north_temp = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.plot(northern_hem.iloc[:,2],regress_values_north_temp,"r-")
plt.annotate(line_eq_north_temp,(0,-35),fontsize=15,color="red")

plt.show()

# Save as .png
plt.savefig('lat_north_temp.png')

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
# Create scatter plot
plt.scatter(southern_hem.iloc[:,2],southern_hem.iloc[:,3])
plt.xlabel("Latitude")
plt.ylabel("Temperature (C)")
plt.title("Latitude vs. Temperature, Southern Hemisphere")

# Calculate and apply linear regression
(slope, intercept, rvalue, pvalue, stderr) = linregress(southern_hem.iloc[:,2],southern_hem.iloc[:,3])
regress_values_south_temp = southern_hem.iloc[:,2] * slope + intercept
line_eq_south_temp = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.plot(southern_hem.iloc[:,2],regress_values_south_temp,"r-")
plt.annotate(line_eq_south_temp,(-45,5),fontsize=15,color="red")

plt.show()

# Save as .png
plt.savefig('lat_south_temp.png')

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
# Create scatter plot
plt.scatter(northern_hem.iloc[:,2],northern_hem.iloc[:,4])
plt.xlabel("Latitude")
plt.ylabel("Humidity")
plt.title("Latitude vs. Humidity, Northern Hemisphere")

# Calculate and apply linear regression
(slope, intercept, rvalue, pvalue, stderr) = linregress(northern_hem.iloc[:,2],northern_hem.iloc[:,4])
regress_values_north_hum = northern_hem.iloc[:,2] * slope + intercept
line_eq_north_hum = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.plot(northern_hem.iloc[:,2],regress_values_north_hum,"r-")
plt.annotate(line_eq_north_hum,(40,35),fontsize=15,color="red")

plt.show()

# Save as .png
plt.savefig('lat_north_hum.png')

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
# Create scatter plot
plt.scatter(southern_hem.iloc[:,2],southern_hem.iloc[:,4])
plt.xlabel("Latitude")
plt.ylabel("Humidity")
plt.title("Latitude vs. Humidity, Southern Hemisphere")

# Calculate and apply linear regression
(slope, intercept, rvalue, pvalue, stderr) = linregress(southern_hem.iloc[:,2],southern_hem.iloc[:,4])
regress_values_south_hum = southern_hem.iloc[:,2] * slope + intercept
line_eq_south_hem = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.plot(southern_hem.iloc[:,2],regress_values_south_hum,"r-")
plt.annotate(line_eq_south_hem,(-25,21),fontsize=15,color="red")

plt.show()

# Save as .png
plt.savefig('lat_south_hum.png')

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
# Create scatter plot
plt.scatter(northern_hem.iloc[:,2],northern_hem.iloc[:,5])
plt.xlabel("Latitude")
plt.ylabel("Cloudiness")
plt.title("Latitude vs. Cloudiness, Northern Hemisphere")

# Calculate and apply linear regression
(slope, intercept, rvalue, pvalue, stderr) = linregress(northern_hem.iloc[:,2],northern_hem.iloc[:,5])
regress_values_north_cloud = northern_hem.iloc[:,2] * slope + intercept
line_eq_north_cloud = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.plot(northern_hem.iloc[:,2],regress_values_north_cloud,"r-")
plt.annotate(line_eq_north_cloud,(40,60),fontsize=15,color="red")

plt.show()

# Save as .png
plt.savefig('lat_north_clouds.png')

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
# Create scatter plot
plt.scatter(southern_hem.iloc[:,2],southern_hem.iloc[:,5])
plt.xlabel("Latitude")
plt.ylabel("Cloudiness")
plt.title("Latitude vs. Cloudiness, Southern Hemisphere")

# Calculate and apply linear regression
(slope, intercept, rvalue, pvalue, stderr) = linregress(southern_hem.iloc[:,2],southern_hem.iloc[:,5])
regress_values_south_cloud = southern_hem.iloc[:,2] * slope + intercept
line_eq_south_cloud = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.plot(southern_hem.iloc[:,2],regress_values_south_cloud,"r-")
plt.annotate(line_eq_south_cloud,(-55,25),fontsize=15,color="red")

plt.show()

# Save as .png
plt.savefig('lat_south_clouds.png')

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
# Create scatter plot
plt.scatter(northern_hem.iloc[:,2],northern_hem.iloc[:,6])
plt.xlabel("Latitude")
plt.ylabel("Wind Speed")
plt.title("Latitude vs. Wind Speed, Northern Hemisphere")

# Calculate and apply linear regression
(slope, intercept, rvalue, pvalue, stderr) = linregress(northern_hem.iloc[:,2],northern_hem.iloc[:,6])
regress_values_north_wind = northern_hem.iloc[:,2] * slope + intercept
line_eq_north_wind = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.plot(northern_hem.iloc[:,2],regress_values_north_wind,"r-")
plt.annotate(line_eq_north_wind,(2,18),fontsize=15,color="red")

plt.show()

# Save as .png
plt.savefig('lat_north_wind.png')

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
# Create scatter plot
plt.scatter(southern_hem.iloc[:,2],southern_hem.iloc[:,6])
plt.xlabel("Latitude")
plt.ylabel("Wind Speed")
plt.title("Latitude vs. Wind Speed, Southern Hemisphere")

# Calculate and apply linear regression
(slope, intercept, rvalue, pvalue, stderr) = linregress(southern_hem.iloc[:,2],southern_hem.iloc[:,6])
regress_values_south_wind = southern_hem.iloc[:,2] * slope + intercept
line_eq_south_wind = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.plot(southern_hem.iloc[:,2],regress_values_south_wind,"r-")
plt.annotate(line_eq_south_wind,(-25,13),fontsize=15,color="red")

plt.show()

# Save as .png
plt.savefig('lat_south_wind.png')